<a href="https://colab.research.google.com/github/paishowstopper/TSAI/blob/main/S7/END_Session7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install torchtext==0.3.1
!pip install google_trans_new

     |████████████████████████████████| 71kB 7.0MB/s 
  Found existing installation: torchtext 0.9.0
    Uninstalling torchtext-0.9.0:
      Successfully uninstalled torchtext-0.9.0


In [2]:
from gensim.parsing.preprocessing import remove_stopwords
import nltk
nltk.download('wordnet')
from nltk.corpus import wordnet
import random
import torch
import pandas as pd
import torch.nn as nn
import torch.nn.functional as F
import os, pickle
import torch.optim as optim
import spacy, math
from torchtext.data import Field, LabelField, Example, Dataset, BucketIterator
import google_trans_new
from google_trans_new import google_translator

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [3]:
def get_synonyms(word):
  synonyms = []
  for syn in wordnet.synsets(word): 
    for l in syn.lemmas():
        synonyms.append(l.name())
  if len(synonyms) > 0:
    return random.choice(synonyms)
  else:
    return None

In [4]:
def random_insertion(sentence, n=3): 
    new_sentence = sentence.split(' ')
    words = remove_stopwords(sentence).split()
    for _ in range(n):
        new_synonym = get_synonyms(random.choice(words))
        if new_synonym is not None:
          new_sentence.insert(random.randrange(len(sentence)+1), new_synonym)
        else:
          new_synonym = ''        
    return ' '.join(new_sentence)

In [5]:
def random_deletion(words, p=0.5): 
    if len(words) == 1: # return if single word
        return words
    remaining = list(filter(lambda x: random.uniform(0,1) > p,words)) 
    if len(remaining) == 0: # if not left, sample a random word
        return [random.choice(words)] 
    else:
        return remaining

In [6]:
def random_swap(sentence, n=5): 
    length = range(len(sentence))
    if n > len(sentence):
      n = len(sentence)
    for _ in range(n):
        idx1, idx2 = random.sample(length, 2)
        sentence[idx1], sentence[idx2] = sentence[idx2], sentence[idx1] 
    return sentence

In [7]:
translator = google_translator()
available_langs = list(google_trans_new.LANGUAGES.keys()) 
trans_lang = random.choice(available_langs) 
  
def google_translate(sentence):
  translations = translator.translate(sentence, lang_tgt=trans_lang) 
  t_text = ''.join([t for t in translations])
  translations_en_random = translator.translate(t_text, lang_src=trans_lang, lang_tgt='en') 
  en_text = ''.join([t for t in translations_en_random])
  return en_text

In [8]:
df = pd.read_csv('tweets.csv')

In [9]:
# Manual Seed
SEED = 43
torch.manual_seed(SEED)

In [10]:
split_length = math.ceil(len(df)*0.85)

In [11]:
train_df = df.iloc[:split_length]
valid_df = df.iloc[split_length:]

In [12]:
augmented_train_df= train_df.sample(n=25,random_state=SEED)
augmented_train_df['tweets'] = augmented_train_df['tweets'].apply(google_translate)
train_df.update(augmented_train_df)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:6397: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[col] = expressions.where(mask, this, that)


In [13]:
augmented_train_df = train_df.sample(n=25, random_state=SEED)
augmented_train_df['tweets'] = augmented_train_df['tweets'].apply(random_insertion)
train_df.update(augmented_train_df)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:6397: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[col] = expressions.where(mask, this, that)


In [14]:
augmented_train_df = train_df.sample(n=25,random_state=SEED)
augmented_train_df['tweets'] = augmented_train_df['tweets'].str.split().apply(random_deletion).apply(' '.join)
train_df.update(augmented_train_df)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:6397: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[col] = expressions.where(mask, this, that)


In [15]:
augmented_train_df= train_df.sample(n=5,random_state=SEED)
augmented_train_df['tweets'] = augmented_train_df['tweets'].str.split().apply(random_swap).apply(' '.join)
train_df.update(augmented_train_df)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:6397: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[col] = expressions.where(mask, this, that)


In [16]:
len(train_df)

1160

In [17]:
Tweet = Field(sequential = True, tokenize = 'spacy', batch_first =True, include_lengths=True)
Label = LabelField(tokenize ='spacy', is_target=True, batch_first =True, sequential =False)

In [18]:
fields = [('tweets', Tweet),('labels',Label)]

In [19]:
train_example = [Example.fromlist([train_df.tweets[i],train_df.labels[i]], fields) for i in range(train_df.shape[0])]

In [20]:
valid_example = [Example.fromlist([valid_df.tweets[i+split_length],valid_df.labels[i+split_length]], fields) for i in range(valid_df.shape[0])]

In [21]:
train = Dataset(train_example, fields)
valid = Dataset(valid_example, fields)

In [22]:
Tweet.build_vocab(train)
Label.build_vocab(train)

In [23]:
print('Size of input vocab : ', len(Tweet.vocab))
print('Size of label vocab : ', len(Label.vocab))
print('Top 10 words appreared repeatedly :', list(Tweet.vocab.freqs.most_common(10)))
print('Labels : ', Label.vocab.stoi)

Size of input vocab :  4540
Size of label vocab :  3
Top 10 words appreared repeatedly : [('Obama', 1047), (':', 811), ('#', 780), ('.', 758), (',', 609), ('"', 585), ('the', 552), ('RT', 524), ('?', 438), ('to', 410)]
Labels :  defaultdict(<function _default_unk_index at 0x7f7dab96e050>, {0.0: 0, 1.0: 1, 2.0: 2})


In [24]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [25]:
train_iterator, valid_iterator = BucketIterator.splits((train, valid), batch_size = 32, 
                                                            sort_key = lambda x: len(x.tweets),
                                                            sort_within_batch=True, device = device)

In [26]:
with open('tokenizer.pkl', 'wb') as tokens: 
  pickle.dump(Tweet.vocab.stoi, tokens)

In [27]:
class classifier(nn.Module):
    
    # Define all the layers used in model
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, dropout):
        
        super().__init__()          
        
        # Embedding layer
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        
        # LSTM layer
        self.encoder = nn.LSTM(embedding_dim, 
                           hidden_dim, 
                           num_layers=n_layers, 
                           dropout=dropout,
                           batch_first=True)
        # try using nn.GRU or nn.RNN here and compare their performances
        # try bidirectional and compare their performances
        
        # Dense layer
        self.fc = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, text, text_lengths):
        
        # text = [batch size, sent_length]
        embedded = self.embedding(text)
        # embedded = [batch size, sent_len, emb dim]
      
        # packed sequence
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths.cpu(), batch_first=True)
        
        packed_output, (hidden, cell) = self.encoder(packed_embedded)
        #hidden = [batch size, num layers * num directions,hid dim]
        #cell = [batch size, num layers * num directions,hid dim]
    
        # Hidden = [batch size, hid dim * num directions]
        dense_outputs = self.fc(hidden)   
        
        # Final activation function softmax
        output = F.softmax(dense_outputs[0], dim=1)
            
        return output

In [28]:
# Define hyperparameters
size_of_vocab = len(Tweet.vocab)
embedding_dim = 300
num_hidden_nodes = 100
num_output_nodes = 3
num_layers = 2
dropout = 0.2

# Instantiate the model
model = classifier(size_of_vocab, embedding_dim, num_hidden_nodes, num_output_nodes, num_layers, dropout = dropout)

In [29]:
print(model)

#No. of trianable parameters
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
    
print(f'The model has {count_parameters(model):,} trainable parameters')

classifier(
  (embedding): Embedding(4540, 300)
  (encoder): LSTM(300, 100, num_layers=2, batch_first=True, dropout=0.2)
  (fc): Linear(in_features=100, out_features=3, bias=True)
)
The model has 1,603,903 trainable parameters


In [30]:

# define optimizer and loss
optimizer = optim.Adam(model.parameters(), lr=2e-4)
criterion = nn.CrossEntropyLoss()

# define metric
def binary_accuracy(preds, y):
    #round predictions to the closest integer
    _, predictions = torch.max(preds, 1)
    
    correct = (predictions == y).float() 
    acc = correct.sum() / len(correct)
    return acc
    
# push to cuda if available
model = model.to(device)
criterion = criterion.to(device)

In [31]:
def train(model, iterator, optimizer, criterion):
    
    # initialize every epoch 
    epoch_loss = 0
    epoch_acc = 0
    
    # set the model in training phase
    model.train()  
    
    for batch in iterator:
        
        # resets the gradients after every batch
        optimizer.zero_grad()   
        
        # retrieve text and no. of words
        tweet, tweet_lengths = batch.tweets   
        
        # convert to 1D tensor
        predictions = model(tweet, tweet_lengths).squeeze()  
        
        # compute the loss
        loss = criterion(predictions, batch.labels)        
        
        # compute the binary accuracy
        acc = binary_accuracy(predictions, batch.labels)   
        
        # backpropage the loss and compute the gradients
        loss.backward()       
        
        # update the weights
        optimizer.step()      
        
        # loss and accuracy
        epoch_loss += loss.item()  
        epoch_acc += acc.item()    
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [32]:
def evaluate(model, iterator, criterion):
    
    # initialize every epoch
    epoch_loss = 0
    epoch_acc = 0

    # deactivating dropout layers
    model.eval()
    
    # deactivates autograd
    with torch.no_grad():
    
        for batch in iterator:
        
            # retrieve text and no. of words
            tweet, tweet_lengths = batch.tweets
            
            # convert to 1d tensor
            predictions = model(tweet, tweet_lengths).squeeze()
            
            # compute loss and accuracy
            loss = criterion(predictions, batch.labels)
            acc = binary_accuracy(predictions, batch.labels)
            
            # keep track of loss and accuracy
            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [33]:
N_EPOCHS = 10
best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
     
    # train the model
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    
    # evaluate the model
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    # save the best model
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'saved_weights.pt')
    
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}% \n')

	Train Loss: 1.034 | Train Acc: 66.55%
	 Val. Loss: 1.018 |  Val. Acc: 68.45% 

	Train Loss: 0.968 | Train Acc: 71.45%
	 Val. Loss: 0.970 |  Val. Acc: 68.90% 

	Train Loss: 0.909 | Train Acc: 72.04%
	 Val. Loss: 0.929 |  Val. Acc: 67.56% 

	Train Loss: 0.856 | Train Acc: 73.48%
	 Val. Loss: 0.884 |  Val. Acc: 68.45% 

	Train Loss: 0.815 | Train Acc: 76.01%
	 Val. Loss: 0.857 |  Val. Acc: 72.02% 

	Train Loss: 0.789 | Train Acc: 77.96%
	 Val. Loss: 0.844 |  Val. Acc: 72.02% 

	Train Loss: 0.770 | Train Acc: 78.72%
	 Val. Loss: 0.839 |  Val. Acc: 73.36% 

	Train Loss: 0.753 | Train Acc: 80.32%
	 Val. Loss: 0.837 |  Val. Acc: 74.26% 

	Train Loss: 0.739 | Train Acc: 81.76%
	 Val. Loss: 0.831 |  Val. Acc: 73.81% 

	Train Loss: 0.727 | Train Acc: 83.53%
	 Val. Loss: 0.827 |  Val. Acc: 74.70% 



In [34]:
#load weights and tokenizer

path='./saved_weights.pt'
model.load_state_dict(torch.load(path));
model.eval();
tokenizer_file = open('./tokenizer.pkl', 'rb')
tokenizer = pickle.load(tokenizer_file)

#inference 

nlp = spacy.load('en')

def classify_tweet(tweet):
    
    categories = {0: "Negative", 1:"Positive", 2:"Neutral"}
    
    # tokenize the tweet 
    tokenized = [tok.text for tok in nlp.tokenizer(tweet)] 
    # convert to integer sequence using predefined tokenizer dictionary
    indexed = [tokenizer[t] for t in tokenized]        
    # compute no. of words        
    length = [len(indexed)]
    # convert to tensor                                    
    tensor = torch.LongTensor(indexed).to(device)   
    # reshape in form of batch, no. of words           
    tensor = tensor.unsqueeze(1).T  
    # convert to tensor                          
    length_tensor = torch.LongTensor(length)
    # Get the model prediction                  
    prediction = model(tensor, length_tensor)

    _, pred = torch.max(prediction, 1) 
    
    return categories[pred.item()]

In [45]:
classify_tweet("A valid explanation for why Trump won't let women on the golf course.")

'Negative'